In [11]:
class foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def T(self):
        return foo(self.y, self.x)
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
f = foo(10, 20)
f.T()

(20, 10)

In [3]:
from typing import TypeVar, Union

n = TypeVar("n")
w = TypeVar("w", *[int, float])

class Edge:
    """
    Simple class that implements weighed edge of a graph

    Attributes
    ----------
    source: n
        Source of the edge
    destination: n
        Destination of the edge
    weight: w
        weight of the edge
    """
    def __init__(self, source: n, destination: n, weight: w):
        self.source = source
        self.destination = destination
        self.weight = weight
    
    def get_source(self):
        return self.source
    
    def get_destination(self):
        return self.destination
    
    def T(self):
        return Edge(self.destination, self.source, self.weight) 
    
    def decompose(self):
        return [self.source, self.destination]

    
class Shortcut:
    def __init__(self, edge1, edge2):
        self.e1 = edge1
        self.e2 = edge2
        self.weight = edge1.weight + edge2.weight
    
    def get_source(self):
        return self.e1.get_source()
    
    def get_destination(self):
        return self.e2.get_destination()
    
        
    def T(self):
        return Shortcut(self.e2.T(), self.e1.T())
    
    def decompose(self):
        path = self.e1.decompose()
            
        path.extend(self.e2.decompose()[1:])
            
        return path


        
class Vertex:
    """
    Class that implements a graph vertex, the class uses a dictionary to store
    the outward connection from this Vertex to others
    
    Attributes
    ----------
    name: n
        Name of the vertex
    connections: dict[n, w]
        Dictionary holding destination:weight pairs
    
    Methods
    -------
    add(destination: n, weight: w)
        Adds a connection to the Vertex, 
        i.e. it adds a destination:weight pair to the connection dictionary
        
    remove(destination: n):
        Removes a connection from the vertex
        
    get_connections():
        Returns the connections dictionary   
    """
    
    def __init__(self, name: n, edges: list = None):
        """
        Class constructor, given a name and a list of edges builds a new vertex
        
        Parameters
        ----------
        name: n
            Name of the vertex
            
        edges: list[Edge], Optional
            List of members of the Edge class, used to build the outward
            connections of the edge
            Default: None i.e. builds a vertex with no connections
        """
        
        self.name = name
        if edges is None:
            self.connections = dict()
        else:
            self.connections = dict([(edge.destination, edge) for edge in edges if edge.source == name])
     
    
    def add(self, destination: n, weight: w):
        """
        Adds a connection to the Vertex
        
        Parameters
        ----------
        destination: n
            Destination of the connection
        weight: w
            Weight of the connection
        """
        self.connections[destination] = Edge(self.name, destination, weight)
        
    
    def add_shortcut(self, shortcut: Shortcut):
        self.connections[shortcut.e2.get_destination()] = shortcut
        
        
    def remove(self, destination: n):
        """
        Removes a connection from the vertex
        
        Parameters
        ----------
        destination: n
            Destination of the connection to remove
        """
        try:
            self.connections.pop(destination)
        except KeyError:
            pass
        
        
    def get_connections(self):
        """Returns the connections dictionary"""
        return self.connections
    
    
    def has_connection(destination: n):
        try:
            a = self.connections[destination]
        except KeyError:
            return False
        
        return True
        
        
    def get_weight(self, destination: n) -> Union[float, int, None]:
        """
        Returns the weight of the connection from this vertex to destination
        if the connection is not present returns None
        
        Parameters
        ----------
        destination: n
            Destination of the connection to get the weight
            
        Returns
        -------
        connection_weight: Union[float, int, None]
            Weight of the connection from this vertex to destination,
            if the connection is not present returns None
        """
        try:
            return self.connections[destination].weight
        except KeyError:
            return None

        
    def __repr__(self):
        """Overload of the print function"""
        return f"{self.name} : {self.get_connections()}"
        

class Graph:
    """
    Implements a weighed graph structure, can be either directed or undirected
    
    Attributes
    ----------
    _graph: dict[n, Vertex]
        Stores the graph connection in a ordered list-like fashion: given a
        node name it returns an instance of the class Vertex
    _directed: bool
        True if the graph is directed, False otherwise
    
    Methods
    -------
    add_vertex(vertex_name: n)
        Adds a new vertex to the graph
        
    remove_vertex(vertex_name: n)
        Removes a vertex from the graph
    
    contains_vertex(vertex_name: n) -> bool
        Returns True if the graph contains the given vertex, False otherwise
        
    add_edge(source: n, destination: n, weight: w = 1.)
        Adds an edge to the graph, if the vertexes containing source and
        destination are not present it adds them as well
            
    remove_edge(source: n, destination: n)
        Removes the edge (source, destination) from the graph

    contains_edge(source: n, destination: n) -> bool
        Returns True if the graph contains the edge (source, destination),
        False otherwise
        
    Adj(vertex_name: n) -> list[n]
        Returns the adjacent vertexes of the given vertex
    
    V() -> list[Vertex]
        Returns a list of all the vertexes present in the graph
    
    E() -> list[(n, n)]
        Returns a list of all the edges present in the graph
    """
    
    def __init__(self, edges=None, directed: bool = True):
        """
        Class constructor, creates an empty graph
        
        Parameters
        ----------
        directed: bool
            If True it creates a directed graph, if False an undirected graph
            Default: True
        """
        
        self._graph = dict()
        self._directed = directed
        if edges is not None:
            [self.add_edge(edge) for edge in edges]
    
    
    def add_shortcuts(self):
        # Get vertex importances
        try:
            importances = [(v.importance, v) for v in self.V()]
        except AttributeError:
            warn("The graph has vertexes without importance, exiting", RuntimeWarning)
            return
        
        # Sort by increasing importance
        importances.sort(key = lambda x, y: x[0]<y[0])
        
        for v in self.V():
            add_shortcut(v)
        
    def add_shortcut(self, v: Vertex):
        parents = self.get_parents(vertex)
        childs = self.get_childs(vertex)
        
        try:
            for p in parents:
                if p.importance < v.importance:
                    continue
                for c in childs:
                    if c.importance < v.importance:
                        continue

                    w = p.get_weight(v) + v.get_weight(c)

                    if p.get_weight(c) is None or p.get_weight(c) > w:
                        shortcut = Shortcut()
                        self.add_edge(p.name, c.name, w)
        
        except AttributeError:
            warn("The graph has vertexes without importance, exiting", RuntimeWarning)
            return
        
    
    def get_parents(self, vertex: Vertex):
        return [v for v in self.V() if vertex.name in v.get_connections().keys()]
    
    
    def get_childs(self, vertex: Vertex):
        return v.get_connections().keys()
    
    
    def add_vertex(self, vertex_name: n):
        """
        Adds a new vertex to the graph
        
        Parameters
        ----------
        vertex_name: n
            Name of the vertex to be added
        """
        new_vertex = Vertex(vertex_name)
        self._graph[new_vertex.name] = new_vertex
       
    
    def remove_vertex(self, vertex_name: n):
        """
        Removes a vertex from the graph
        
        Parameters
        ----------
        vertex_name: n
            Name of the vertex to be removed
        """
        # Remove incoming connections
        for vertex in self._graph.values():
            vertex.remove(vertex_name)
        
        # Remove the vertex
        try:
            self._graph.pop(vertex_name)
        except KeyError:
            pass
    
    
    def contains_vertex(self, vertex_name: n):
        """Returns True if the graph contains the given vertex, False otherwise"""
        return vertex_name in self._graph.keys()
    
        
    def add_edge(self, source: n, destination: n, weight: w = 1.):
        """
        Adds an edge to the graph, if the vertexes containing source and
        destination are not present it adds them as well
        
        Parameters
        ----------
        source: n
            Source of the edge to be added
        
        destination: n
            Destination of the edge
        
        weight: w, Optional
            Weight of the connection, Default=1.
        """
        
        if not self.contains_vertex(destination):
            self.add_vertex(destination)
        if not self.contains_vertex(source):
            self.add_vertex(source)
        
        self._graph[source].add(destination, weight)
            
        if not self._directed:
            self._graph[destination].add(source, weight)
            
            
    def get_vertex(self, vertex_name: n) -> Union[Vertex, None]:
        """
        Returns the vertex with the given name
        
        Parameters
        ----------
        vertex_name: n
            Name of the vertex to be found
            
        Returns
        -------
        Union(Vertex, None):
            A vertex if vertex_name is present in the graph, otherwise None
        """
        try:
            return self._graph[vertex_name]
        except KeyError:
            return None
        
    def remove_edge(self, source: n, destination: n):
        """
        Removes the edge (source, destination) from the graph
        """
        self._graph[source].remove(destination)
        if not self._directed:
            self._graph[destination].remove(source)
    
    
    def contains_edge(self, source: n, destination: n) -> bool:
        """ 
        Returns True if the graph contains the edge (source, destination),
        False otherwise
        """
        if not contains_vertex(source):
            return False
        if not contains_vertex(destination):
            return False
        
        return self._graph[source].has_connection(destination)

      
        
    def Adj(self, vertex_name: n) -> list:
        """Returns the adjacent vertexes of the given vertex"""
        return self._graph[vertex_name].get_connections().keys()
    
    
    def V(self) -> list:
        """Returns a list of all the vertexes present in the graph"""
        return list(self._graph.values())
    
    
    def E(self) -> list:
        """Returns a list of all the edges present in the graph"""
        res = []
        for v in self.V():
            res.extend([i for i in v.get_connections().values()])
        return res
    
    
    def T():
        """Returns a transposed version of the graph"""
        edges = []
        G = Graph()
        for edge in self.E():
            if isinstance(edge, Shortcut):
                G.add_shortcut(edge.source, edge.destination, edge.weight)
            G.add_edge(edge.source, edge.destination, edge.weight)
        
        f
        return Graph(edges)
            
    
    def __repr__(self):
        """Overload of the print function"""
        s = ""
        for v in self.V():
            s += f"{v.__repr__()}\n"
            
        return s

IndentationError: expected an indented block (<ipython-input-3-69bb2dcd5c37>, line 32)

In [ ]:
def init_sssp(G, s):
    u = G.get_vertex(s)
    if u is None:
        message = f"The vertex{s} is not present in the graph, exiting"
        warn(message, RuntimeWarning)
        return
    
    for v in G.V():
        v.d = inf
        v.pred = None
        
    u.d = 0.
    
    
def relax(Q, u, v):
    w = u.get_weight(v.name)
    if v.d > u.d + w:
        v.d = u.d + w
        v.pred = u
        Q.decrease_key((v.d, v))

        
def Bidirectional_Dijkstra(G: Graph, s: n, d: n):
    G.add_shortcuts()
    G_transpose = G.T
    init_sssp(G, s)
    Q = Binary_heap(G.V(), total_order=lambda x, y: x.d < y.d, dict_key=lambda x: x.name)
    while not Q.is_empty():
        u = Q.remove_min()
        for v in G.Adj(u.name):
            v = G.get_vertex(v)
            relax(Q, u, v)

In [39]:
G = Graph()
G.add_edge("A", "C", 1.)

G.E()

[('A', 'C')]

In [40]:
G.add_edge("C", "B")

In [41]:
G

C : {'B': 1.0}
A : {'C': 1.0}
B : {}

In [42]:
G.E()

[('C', 'B'), ('A', 'C')]

In [27]:
G.remove_vertex("A")
G.E()

[('C', 'B')]

In [43]:
G.add_vertex("D")
G.V()

dict_values([C : {'B': 1.0}, A : {'C': 1.0}, B : {}, D : {}])

In [54]:
list(G.V())[0].connections

{'B': 1.0}

In [30]:
G.add_edge("D", "A")

In [35]:
G.E()

[('C', 'B'), ('D', 'A')]

In [37]:
G.Adj("D")

{'A': 1.0}